In [1]:
# !pip install htmldate
# !pip install twython
# !pip3 install newspaper3k

In [8]:
import pandas as pd
import numpy as np
import time
import twython
import requests
import nltk
import warnings
warnings.filterwarnings('ignore')

from htmldate import find_date
from tqdm import tqdm
from bs4 import BeautifulSoup
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')
from newspaper import Article

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\KyThuat88\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [15]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [16]:
from selenium.webdriver.common.by import By
def get_newslinks(company, page_number):
    """For a given URL, scroll to relevant section to load appropriate HTML into driver,
    iterate through all articles on page and append article URLs to a list

    :param company: name of company to scrape articles for
    :param page_number: page number on news website to iterate over 

    :return: list of articles URLs
    """
    
    url = f"https://www.investing.com/equities/{company}-news/{page_number}"
    driver.get(url)

    href = []

    # scroll all the way to the bottom 

    old_position = 0
    new_position = None

    while new_position != old_position:
        # Get old scroll position
        old_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                " window.pageYOffset : (document.documentElement ||"
                " document.body.parentNode || document.body);"))
        # Sleep and Scroll
        time.sleep(1)
        driver.execute_script((
                "var scrollingElement = (document.scrollingElement ||"
                " document.body);scrollingElement.scrollTop ="
                " scrollingElement.scrollHeight;"))
        # Get new position
        new_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                " window.pageYOffset : (document.documentElement ||"
                " document.body.parentNode || document.body);"))
        
    cleaned_links = []

    # Iterate through all the articles on the page
    for article_number in range(1,11): 
        article = driver.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div[2]/div[2]/div[1]/div[2]/ul/li[{article_number}]/article')
        #article = driver.find_element(By.XPATH, f'/html/body/div[5]/section/div[8]/article[{article_number}]')
        article_html = article.get_attribute('innerHTML')
        soup = BeautifulSoup(article_html, "lxml")
        # for link in soup.find_all('a'): 
        #     # Get the href
        #     partial_link = link.get('href')
        #     if 'https' in partial_link: 
        #         cleaned_links.append(partial_link)
        #     # Some links are 'internal' to the page and don't have https in them. The web page must be appended to these links
        #     elif partial_link[0] == '/': 
        #         cleaned_links.append("https://www.investing.com/"+partial_link) 
        for link in soup.find_all('a'):
            # Get the href
            partial_link = link.get('href')
            if partial_link:
                if 'https' in partial_link: 
                    cleaned_links.append(partial_link)
                # Some links are 'internal' to the page and don't have https in them. The web page must be appended to these links
                elif partial_link[0] == '/': 
                    cleaned_links.append('https://www.investing.com'+partial_link) 

    return np.unique(cleaned_links)

In [17]:
all_company_urls = []
for page in range(11,100):
    results = get_newslinks('apple-computer-inc', page)
    all_company_urls.extend(results)
    print(len(all_company_urls))

15
28
41
55
67
81
94
110
121
132
143
158
171
182
196
206
219
233
246
260
273
286
301
316
328
344
359
373
387
403
416
430
444
457
473
490
508
520
535
550
562
578
592
605
619
632
648
662
679
692
704
717
731
749
763
777
790
804
817
831
842
856
871
884
900
919
935
950
964
978
995
1009
1020
1033
1048
1060
1072
1085
1098
1112
1128
1139
1150
1164
1177
1190
1204
1220
1233


In [23]:
# AstraZeneca stock ticker
ticker = 'AAPL'
# Create a DataFrame to populate while iterating
article_sentiments = pd.DataFrame({'ticker':[],
                                'publish_date':[],
                                'title': [],
                                'body_text': [],
                                'url':[],
                                'neg':[],
                                'neu':[], 
                                'pos':[], 
                                'compound':[]})
# Loop over all the articles
for link in all_company_urls:
      article = Article(link)
      article.download()
      
      try:
          article.parse()
          text = article.text

      except: 
          print("I didn't get this")
          continue

      # Initialise sentiment analyser    
      sid = SentimentIntensityAnalyzer()
      # Get positive, negative, neutral and compound scores
      polarity = sid.polarity_scores(text)

      tmpdic = {'ticker': ticker, 'publish_date': find_date(link), 'title': article.title, 'body_text': article.text, 'url': link}
      # Update ticker with the new entry polarity
      tmpdic.update(polarity)
      tmpdic_df = pd.DataFrame([tmpdic])
      # tmpdic now has all keys and values needed to populate the DataFrame
      article_sentiments = pd.concat([article_sentiments,tmpdic_df],ignore_index = True)
      #article_sentiments= article_sentiments.append(pd.DataFrame(tmpdic, index=[0]))
      article_sentiments.reset_index(drop=True, inplace=True)

I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't get this
I didn't g

In [24]:
article_sentiments

,ticker,publish_date,title,body_text,url,neg,neu,pos,compound
0,AAPL,2024-04-12,"Morning Bid: Eyes switch to earnings, China tr...",A look at the day ahead in U.S. and global mar...,https://www.investing.com/news/commodities-new...,0.070,0.847,0.083,0.9439
1,AAPL,2024-04-11,US quarterly earnings to feature big growth in...,By Caroline Valetkevitch\n\nNEW YORK (Reuters)...,https://www.investing.com/news/economy/us-quar...,0.029,0.840,0.131,0.9937
2,AAPL,2024-04-13,Apple denies violating US court order in Epic ...,By Mike Scarcella\n\n(Reuters) - IPhone maker ...,https://www.investing.com/news/stock-market-ne...,0.084,0.856,0.059,0.1779
3,AAPL,2024-04-13,Apple denies violating US court order in Epic ...,By Mike Scarcella\n\n(Reuters) - IPhone maker ...,https://www.investing.com/news/stock-market-ne...,0.084,0.856,0.059,0.1779
4,AAPL,2024-04-12,Apple earnings preview: Goldman says but the p...,"Apple's stock surged Thursday, fueled by a Blo...",https://www.investing.com/news/stock-market-ne...,0.039,0.852,0.109,0.9865
...,...,...,...,...,...,...,...,...,...
1074,AAPL,2023-11-14,Google wraps up defense in major antitrust law...,Google (NASDAQ: ) has concluded its defense in...,https://www.investing.com/news/stock-market-ne...,0.028,0.866,0.106,0.9756
1075,AAPL,2023-11-13,Google's expert in US antitrust trial defends ...,By Diane Bartz\n\nWASHINGTON (Reuters) - The m...,https://www.investing.com/news/stock-market-ne...,0.048,0.880,0.072,0.6449
1076,AAPL,2023-11-13,Google's expert in US antitrust trial defends ...,By Diane Bartz\n\nWASHINGTON (Reuters) - The m...,https://www.investing.com/news/stock-market-ne...,0.048,0.880,0.072,0.6449
1077,AAPL,2023-11-14,Huawei's China sales up 83% in October as Mate...,Risk Disclosure: Trading in financial instrume...,https://www.investing.com/news/stock-market-ne...,0.076,0.893,0.032,-0.8437


In [25]:
article_sentiments.to_pickle("apple_article_sentiments.pkl")

In [26]:
article_sentiments.to_csv("apple_article_sentiments.csv", sep=',', encoding='utf-8', header=True)

In [27]:
with open('apple_urls.txt', 'w') as f:
    for link in all_company_urls:
        f.write("%s\n" % link)